# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face
from dotenv import load_dotenv
from huggingface_hub import login
import os

load_dotenv(override=True)
print(f"WANDB_PROJECT: {os.getenv('WANDB_PROJECT')}")
print(f"WANDB_LOG_MODEL: {os.getenv('WANDB_LOG_MODEL')}")
print(f"BLARGH: {os.getenv('BLARGH')}")

login(token=os.getenv("HF_TOKEN"))



# for convenience you can create an environment variable containing your hub token as HF_TOKEN

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


WANDB_PROJECT: smol-course-smehmood
WANDB_LOG_MODEL: checkpoint
BLARGH: None


In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
base_model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=base_model, tokenizer=tokenizer)

# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-smehmood"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [5]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [6]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [45]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
    eval_dataset=ds["test"],
    report_to="wandb"
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=base_model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

TypeError: SFTConfig.__init__() got an unexpected keyword argument 'eval_dataset'

## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [44]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

  0%|          | 0/1000 [00:00<?, ?it/s]

{'loss': 0.3413, 'grad_norm': 1.4055887460708618, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.02}
{'loss': 0.3388, 'grad_norm': 1.4032888412475586, 'learning_rate': 4.9e-05, 'epoch': 0.04}
{'loss': 0.3744, 'grad_norm': 2.010791063308716, 'learning_rate': 4.85e-05, 'epoch': 0.05}
{'loss': 0.3675, 'grad_norm': 2.2714316844940186, 'learning_rate': 4.8e-05, 'epoch': 0.07}
{'loss': 0.354, 'grad_norm': 1.6922193765640259, 'learning_rate': 4.75e-05, 'epoch': 0.09}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.2041603326797485, 'eval_runtime': 3.3033, 'eval_samples_per_second': 36.024, 'eval_steps_per_second': 4.541, 'epoch': 0.09}
{'loss': 0.3844, 'grad_norm': 1.684668779373169, 'learning_rate': 4.7e-05, 'epoch': 0.11}
{'loss': 0.4101, 'grad_norm': 1.7317347526550293, 'learning_rate': 4.6500000000000005e-05, 'epoch': 0.12}
{'loss': 0.3781, 'grad_norm': 1.6600823402404785, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.14}
{'loss': 0.3568, 'grad_norm': 1.9436765909194946, 'learning_rate': 4.55e-05, 'epoch': 0.16}
{'loss': 0.3513, 'grad_norm': 1.9780144691467285, 'learning_rate': 4.5e-05, 'epoch': 0.18}


  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.1856260299682617, 'eval_runtime': 4.4145, 'eval_samples_per_second': 26.956, 'eval_steps_per_second': 3.398, 'epoch': 1.77}


wandb: Adding directory to artifact (./sft_output/checkpoint-1000)... Done. 2.1s


{'train_runtime': 508.5676, 'train_samples_per_second': 7.865, 'train_steps_per_second': 1.966, 'train_loss': 0.4696823668479919, 'epoch': 1.77}


ValueError: You have set `args.eval_strategy` to IntervalStrategy.STEPS but you didn't pass an `eval_dataset` to `Trainer`. Either set `args.eval_strategy` to `no` or pass an `eval_dataset`. 

In [39]:
import wandb
trainer.push_to_hub(tags=finetune_tags)

NameError: name 'wandb' is not defined

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [42]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
fine_tuned_model = AutoModelForCausalLM.from_pretrained(f"./sft_output").to(device)
outputs = fine_tuned_model.generate(**inputs, max_new_tokens=100)
print("After training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

After training:
user
Write a haiku about programming
assistant
I'm a language model, I'm a language model, I'm a language model. I'm a language model. I'm a language model. I'm a language model. I'm a language model. I'm a language model. I'm a language model. I'm a language model. I'm a language model. I'm a language model. I'm a language model. I'm a language model. I'm a language model. I'm a language model.


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.